In [30]:
# langChainFuncs.py

from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.chains import PALChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import PromptTemplate, LLMChain

import streamlit as st
import os 
from dateutil import parser
import datetime
import calendar
import random
import json
from faker import Faker

In [23]:

fake = Faker()
today = datetime.datetime.now()
hours = (9, 18)   # open hours

######### utility functions for scheduling

# create random schedule
def createSchedule(daysAhead=5, perDay=5):
    schedule = {}
    for d in range(0, daysAhead):
        date = (today + datetime.timedelta(days=d)).strftime('%m/%d/%y')
        schedule[date] = {}

        for h in range(0, perDay -d):
            hour = random.randint(hours[0], hours[1])
            if hour not in schedule[date]:
                schedule[date][hour] = fake.name()
                
    return schedule

# get available times for a date
def getAvailTimes(date, num=10):
    schedule = loadSchedule()

    if '/' not in date or 'mm' in date:
        return 'date parameter must be in format: mm/dd/yy'

    if date not in schedule:
        return 'that day is entirely open, all times are available'

    hoursAvail = 'hours available on %s are ' % date

    for h in range(hours[0], hours[1]):
        if str(h) not in schedule[date]:
            hoursAvail += str(h) +':00, '
            num -= 1
            if num == 0:
                break
    
    if num > 0:
        hoursAvail = hoursAvail[:-2] +' - all other times are reserved'
    else:
        hoursAvail = hoursAvail[:-2]
        
    return hoursAvail

# schedule available time
def scheduleTime(dateTime):
    schedule = loadSchedule()

    date, time = dateTime.split(',')
    
    if not date or not time:
        return "sorry parameters must be date and time comma separated, for example: `12/31/23, 10:00` would be the input if for Dec 31'st 2023 at 10am"

    # get hours
    if ':' in time:
        timeHour = int(time[:time.index(':')])
        print(timeHour)
        
        if timeHour not in schedule[date]:
            if timeHour >= hours[0] and timeHour <= hours[1]:
                schedule[date][timeHour] = fake.name()
                saveSchedule(schedule)
                print('Updated schedule json...')
                return 'thank you, appointment scheduled for %s under name %s' % (time, schedule[date][timeHour])
            else:
                return '%s is after hours, please select a time during business hours' % time
        else:
            return 'sorry that time (%s) on %s is not available' % (time, date)
    else:
        return '%s is not a valid time, time must be in format hh:mm'
    
# save schedule json
def saveSchedule(schedule):
    with open('schedule.json', 'w') as f:
        json.dump(schedule, f)
    
# load schedule json
def loadSchedule():
    global schedule
    
    with open('schedule.json') as json_file:
        return json.load(json_file)

# get today's date
def todayDate():
    return today.strftime('%m/%d/%y')

# get day of week for a date (or 'today')
def dayOfWeek(date):
    if date == 'today':
        return calendar.day_name[today.weekday()]
    else:
        try:
            theDate = parser.parse(date)
        except:
            return 'invalid date format, please use format: mm/dd/yy'
        
        return calendar.day_name[theDate.weekday()]
    
#########

In [24]:
os.environ['OPENAI_API_KEY'] = 'YOUR API KEY'

llm = OpenAI(temperature=0, verbose=True)
#llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0, verbose=True)

pal_chain = PALChain.from_math_prompt(llm, verbose=True)


tools = [
    Tool(
        name = "today's date",
        func = lambda string: todayDate(),
        description="use to get today's date",
        ),
    Tool(
        name = "day of the week",
        func = lambda string: dayOfWeek(string),
        description="use to get the day of the week, input is 'today' or date using format mm/dd/yy",
        ),
    Tool(
        name = 'available appointments',
        func = lambda string: getAvailTimes(string),
        description="Use to check on available appointment times for a given date. The input to this tool should be a string in this format mm/dd/yy. This is the only way for you to answer questions about available appointments. This tool will reply with available times for the specified date in 24hour time, for example: 15:00 and 3pm are the same.",
        ),
    Tool(
        name = 'schedule appointment',
        func = lambda string: scheduleTime(string),
        description="Use to schedule an appointment for a given date and time. The input to this tool should be a comma separated list of 2 strings: date and time in format: mm/dd/yy, hh:mm, convert date and time to these formats. For example, `12/31/23, 10:00` would be the input if for Dec 31'st 2023 at 10am",
        ),
    Tool(
        name = "PAL",
        func = pal_chain.run,
        description = "useful for when you need to answer questions about math or word problems or date comparisons"
        )
    ]


In [25]:
memory = ConversationBufferMemory(memory_key="chat_history")
agent_chain = initialize_agent(tools, llm, agent='zero-shot-react-description', memory=memory, verbose=True)

In [26]:
user_input = "What's the date tomorrow?"
agent_chain.run(input=user_input)



> Entering new AgentExecutor chain...
 I need to know what today's date is first.
Action: today's date
Action Input: 
Observation: 04/12/23
Thought: I need to add one day to the date
Action: PAL
Action Input: 04/12/23 + 1 day

> Entering new PALChain chain...
def solution():
    """04/12/23 + 1 day"""
    day = 1
    date = "04/12/23"
    date_list = date.split("/")
    month = int(date_list[0])
    day_of_month = int(date_list[1])
    year = int(date_list[2])
    day_of_month += day
    if day_of_month > 30:
        day_of_month -= 30
        month += 1
    if month > 12:
        month -= 12
        year += 1
    result = f"{month:02d}/{day_of_month:02d}/{year:02d}"
    return result

> Finished chain.

Observation: 04/13/23
Thought: I now know the final answer
Final Answer: Tomorrow's date is 04/13/23.

> Finished chain.


"Tomorrow's date is 04/13/23."

In [27]:
user_input = "what times are available today?"
agent_chain.run(input=user_input)



> Entering new AgentExecutor chain...
 I need to check the available appointments tool
Action: available appointments
Action Input: today's date
Observation: date parameter must be in format: mm/dd/yy
Thought: I need to convert today's date to the correct format
Action: today's date
Action Input: none
Observation: 04/12/23
Thought: I now have the correct date format
Action: available appointments
Action Input: 04/12/23
Observation: hours available on 04/12/23 are 10:00, 15:00, 16:00, 17:00 - all other times are reserved
Thought: I now know the final answer
Final Answer: The available times for today (04/12/23) are 10:00, 15:00, 16:00, 17:00.

> Finished chain.


'The available times for today (04/12/23) are 10:00, 15:00, 16:00, 17:00.'

In [28]:
user_input = "what times are available tomorrow?"
agent_chain.run(input=user_input)



> Entering new AgentExecutor chain...
 I need to check the available appointments tool
Action: available appointments
Action Input: tomorrow's date in mm/dd/yy format
Observation: date parameter must be in format: mm/dd/yy
Thought: I need to get tomorrow's date
Action: today's date
Action Input: none
Observation: 04/12/23
Thought: I now have tomorrow's date
Action: available appointments
Action Input: 04/13/23
Observation: hours available on 04/13/23 are 9:00, 15:00, 16:00 - all other times are reserved
Thought: I now know the final answer
Final Answer: The available times for tomorrow (04/13/23) are 9:00, 15:00, and 16:00.

> Finished chain.


'The available times for tomorrow (04/13/23) are 9:00, 15:00, and 16:00.'

In [31]:
user_input = "what day of the week is tomorrow?"
agent_chain.run(input=user_input)



> Entering new AgentExecutor chain...
 I need to know what today's date is first
Action: today's date
Action Input: none
Observation: 04/12/23
Thought: I need to use the day of the week tool to get the day of the week for tomorrow
Action: day of the week
Action Input: 04/13/23
Observation: Thursday
Thought: I now know the final answer
Final Answer: Tomorrow is Thursday.

> Finished chain.


'Tomorrow is Thursday.'

In [16]:
user_input = "what times are available the day after tomorrow?"
agent_chain.run(input=user_input)



> Entering new AgentExecutor chain...
 I need to figure out what date the day after tomorrow is
Action: today's date
Action Input: none
Observation: 04/12/23
Thought: I need to check what times are available for that date
Action: available appointments
Action Input: 04/12/23
Observation: hours available on 04/12/23 are 10:00, 15:00, 16:00, 17:00 - all other times are reserved
Thought: I now know the final answer
Final Answer: The times available the day after tomorrow are 10:00, 15:00, 16:00, and 17:00.

> Finished chain.


'The times available the day after tomorrow are 10:00, 15:00, 16:00, and 17:00.'

In [10]:
loadSchedule()

{'04/11/23': {'15': 'Shane Clark',
  '9': 'Brittany Barber',
  '11': 'Albert Wong',
  '12': 'Julie Lopez',
  '14': 'John Walker',
  '17': 'Dennis Wright',
  '13': 'James Leonard',
  '16': 'Natasha Frazier',
  '10': 'Robert Powell'},
 '04/12/23': {'12': 'Richard Ramos',
  '11': 'Jacob Garza',
  '9': 'Robin Spence',
  '13': 'Rachel Holmes',
  '14': 'Steven Robinson'},
 '04/13/23': {'12': 'Brandy Hernandez',
  '14': 'Christopher Robinson',
  '17': 'Christy Todd',
  '13': 'Bruce Jones',
  '11': 'Mark Joyce',
  '10': 'Matthew Whitehead'},
 '04/14/23': {'16': 'Stephen Smith',
  '17': 'Shannon Murillo',
  '13': 'Sarah Munoz',
  '15': 'Nathan Parker'},
 '04/15/23': {'12': 'Brittany Brown', '15': 'Rebecca Graham'}}